# Regresión Lineal utilizando `sklearn`: predicción de precios de casas

En este notebook vamos a calcular modelos de regresión lineal para predecir los precios de casas utilizando este dataset disponible en Kaggle: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/.

En primer lugar, cargamos las librerías `numpy` y `pandas` y listamos los ficheros que tenemos en el directorio actual para comprobar que tenemos los ficheros de datos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.listdir("./"))

Cargamos los datos de entrenamiento (`train.csv`) y también los datos de test que trae el dataset (`test.csv`). Los datos de test no tienen la variable que queremos predecir (`SalePrice`) ya que se trata  de un dataset para una competición.

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

print("Train size: ", train.shape)
print("Test size: ", test.shape)

Mostramos las primeras filas de cada dataset:

In [ ]:
print(train.head())
print('**'* 50)
print(test.head())

Mostramos información básica acerca de cada dataset utilizando las funciones `info()` y `describe()`.

In [ ]:
print(train.info())
print('**'* 50)
print(train.describe())
print('**'* 50)
print(test.info())
print('**'* 50)
print(test.describe())

Repasa la descripción del dataset en Kaggle (https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) para entender qué información nos ofrece cada columna.                    

## Visualización de datos

La variable que queremos predecir es el precio de venta, que se encuentra en la columna `SalePrice`. Utilizamos la función `displot` (https://seaborn.pydata.org/generated/seaborn.displot.html#seaborn.displot) de la librería seaborn para visualizar la distribución de esta variable.

In [ ]:
print(train['SalePrice'].describe())

sns.displot(train['SalePrice'], kind="hist")

Puedes explorar otras variables numéricas del dataset utilizando la misma representación.

Otra visualización útil de este tipo de datos es crear un mapa de calor para visualizar la correlación entre todas las las variables numéricas del dataset.

In [ ]:
plt.figure(figsize=(30,8))
sns.heatmap(train.corr(), cmap='coolwarm', annot=True)
plt.show()

Analiza esta tabla, fijándote especialmente en la última fila (o columna) donde se pueden ver las correlaciones con la variable que queremos predecir.

## Preprocesado de datos

### Selección de variables predictoras

Para reducir la cantidad de variables, vamos a quedarnos con aquellas que más correlación tienen con la variable que queremos predecir. Este valor se define en la variable `correlation_threshold`.

In [ ]:
correlation_threshold = 0.35

corr = train.corr()

columns = corr[corr['SalePrice'] > correlation_threshold].index

print(columns)

train.filtered = train[columns]

print("Train size: ", train.filtered.shape)

print(train.filtered.head())

test.filtered = test[columns.drop('SalePrice')]

print("Test size: ", test.filtered.shape)

print(test.filtered.head())

### Visualización de las variables seleccionadas

Podemos visualizar la relación entre cada variable seleccionada y el precio de venta utilizando gráficos de dispersión (o *scatter plots*). Por ejemplo, el siguiente gráfico muestra la relación con `LotFrontage`.

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(x='LotFrontage', y='SalePrice', data=train)
plt.show()

La librería seaborn incluye una función llamada `lmplot` para visualizar un modelo de regresión simple entre dos variables (https://seaborn.pydata.org/generated/seaborn.lmplot.html).

In [ ]:
sns.lmplot(x='LotFrontage', y='SalePrice', data=train)

Algunas variables, en lugar de un rango contunuo muy amplio, se limitan a unos pocos valores. Este es el caso de `GarageCars`, que podemos visualizar utilizando la función `boxplot` de seaborn.

In [ ]:
plt.figure(figsize=(6,6))
sns.boxplot(x='GarageCars',y='SalePrice',data=train)
plt.show()

Analiza el resto de variables utilizando el tipo de gráfico más adecuado (puedes probar los dos y comprobar el resultado).

### Valores perdidos

Vamos a analizar si existen filas con missing data en nuestro dataset.

In [ ]:
total = train.filtered.isnull().sum().sort_values(ascending=False)

print(total)

Aquí podríamos:
- Eliminar las filas con valores perdidos.
- Utilizar alguna estrategia para imputar valores perdidos y poder utilizar todas las filas.

#### Eliminar filas con valores perdidos

Aplicando esta estrategia, simplemente eliminamos las columnas con algún valor perdido utilizando la función `dropna` (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [ ]:
train.filtered.clean = train.filtered.dropna()

total = train.filtered.clean.isnull().sum().sort_values(ascending=False)
print(total)

print(train.filtered.clean.shape)

#### Imputar valores perdidos

Al eliminar filas con valores perdidos reducimos el tamaño del dataset de entrenamiento a 1121 muestras, por lo que puede que estemos perdiendo información útil y valiosa. Para evitar esta pérdida, se pueden aplicar técnicas de imputación de valores perdidos. Básicamente, estas filas consisten en asignar el mismo valor a todos los valores perdidos para una variable (p. ej.: asignar el valor medio). Esta página de sklearn ofrece más información: https://scikit-learn.org/stable/modules/impute.html
        
Vamos a utilizar la clase `SimpleImputer` (https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) para realizar este proceso.

In [ ]:
from sklearn.impute import SimpleImputer

impute_missing=SimpleImputer(missing_values=np.NaN, strategy='mean')
impute_missing.fit(train.filtered)

train.filtered.clean = impute_missing.transform(train.filtered)

train.filtered.clean = pd.DataFrame(data=train.filtered.clean,columns=train.filtered.columns)

total = train.filtered.clean.isnull().sum().sort_values(ascending=False)
print(total)

print(train.filtered.clean.shape)

### Actualización de la correlación y filtrado final



Una vez que hemos limpiado el dataset, podemos calcular de nuevo la correlación con la variable de interés.

In [ ]:
plt.figure(figsize=(30,8))
sns.heatmap(train.filtered.clean.corr(), cmap='coolwarm', annot=True)
plt.show()

Podemos mejorar la visualización anterior de dos maneras: ordenando las variables de mayor a menor correlación con la variable objetivo y cogiendo el subconjunto de `k` variables predictoras con más correlación.

In [ ]:
k = 10 # number of variables for heatmap

plt.figure(figsize=(16,8))
corrmat = train.filtered.clean.corr()

top_cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train.filtered.clean[top_cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, 
    cbar=True, annot=True, square=True, 
    fmt='.2f', annot_kws={'size': 10},
    yticklabels=top_cols.values, xticklabels=top_cols.values)
plt.show()


Podemos filtrar nuevamente los conjuntos de train y test utilizando estas `top_cols`:

In [ ]:
columns = top_cols

print(columns)

train.final = train.filtered.clean[top_cols]

print("Train size: ", train.final.shape)

print(train.final.head())

train.isnull().sum().sort_values(ascending=False).head(20)

test.final = test[columns.drop('SalePrice')]

print("Test size: ", test.final.shape)

print(test.final.head())

test.final.isnull().sum().sort_values(ascending=False).head(20)

## Linear Regression

### División en Train y Validation

Utilizaremos una validación *holdout*, por lo que dividimos el dataset de entrenamiento en dos subcojuntos: 80% de datos para `train` y 20% `validation` (el porcentaje de datos para test se especifica en la variable `test_size`). Observa cómo esta función devuelve 4 subconjuntos.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train.final.drop('SalePrice', axis=1),
    train.final['SalePrice'],
    test_size=0.2,
    random_state=2020
)

## Entrenamiento del modelo

Empezaremos probando el modelo `LinearRegression` (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html). Importamos el modelo que queremos utilizar e invocamos a la función `fit` para ajustar el modelo. Este método se utiliza para entrenar cualquier modelo en sklearn.

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
print(lm)

In [ ]:
print(lm.intercept_)
print(lm.coef_)

## Realizar predicciones para el conjunto de validación

Utilizamos el modelo ajustado para realizar predicciones para el conjunto de validación.

In [ ]:
predictions = lm.predict(X_val)
predictions= predictions.reshape(-1,1)

print(predictions)

Comparamos las predicciones con los valores reales (disponibles en la variable `y_val`). Si las predicciones fuesen perfectas obtendríamos una linea recta, pero como es esperable hay cierto margen de error.

In [ ]:
plt.figure(figsize=(15,8))
plt.scatter(y_val, predictions)
plt.xlabel('Y Val')
plt.ylabel('Predicted Y')
plt.show()

### Evaluación de las predicciones

El paquete `sklearn.metrics` ofrece diferentes métricas para la evaluación de los modelos de clasificación y regresión. En este caso, aplicaremos tres métricas específicas para problemas de regresión:
- Mean Absolute Error (https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error).
- Mean Squared Error (https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error).
- Root Mean Squared Error.
- R^2 score o coeficiente de determinación (https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score).

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(y_val, predictions))
print('MSE:', metrics.mean_squared_error(y_val, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_val, predictions)))
print('R^2:', metrics.r2_score(y_val, predictions))

## Escalado de datos

Como ya hemos visto, en algunos algoritmos (basados en distancias) es necesario escalar los datos de entrada para evitar que ciertas variables dominen el entrenamiento. En el caso de la regresión lineal no es necesario, aunque hay quien prefiere hacerlo.

A la hora de ajustar algunos modelos mediante técnicas de optimización, el hecho de que ciertas variables tengan otra escala pueden hacer que sea más costoso encontrar la solución ópima.

En este apartado veremos cómo escalar datos utilizando la clase `StandardScaler` de skearn (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) y volveremos a calcular la regresión lineal.

En primer lugar, importamos la clase y ajustamos el escalador utilizando solo los datos de la partición entrenamiento.

In [ ]:
from sklearn.preprocessing import StandardScaler

scale = StandardScaler().fit(X_train)

Este objeto `scale` ajustado lo utilizaremos a continuación para escalar tanto la propia partición de entrenamiento como la partición de validación. Esto debe hacerse así para no traspasar información (indirectamente) del conjunto de validación al ajuste del modelo, algo que pasaría si creásemos el objeto de escalado utilizando el conjunto completo de datos de train.

Comenzamos por la partición de entrenamiento, la escalamos y la comparamos con la original.

In [ ]:
X_train.scaled = scale.transform(X_train)
X_train.scaled = pd.DataFrame(data=X_train.scaled, columns=X_train.columns)

print(X_train.describe())
print(X_train.scaled.describe())

Y escalamos del mismo modo la partición de validación. Al entrenar el modelo con datos escalado, significa que debemos escalar también los datos de validación y de test. En un caso real, si tuviésemos el modelo ajustado y quisésemos clasificar una muestra nueva, tendríamos que escalar esta antes de aplicarle el modelo (utilizando el escalado calculado con los datos con los que ajustamos el modelo).

In [ ]:
X_val.scaled = scale.transform(X_val)
X_val.scaled = pd.DataFrame(data=X_val.scaled, columns=X_val.columns)

Y finalmente, repetimos el proceso anterior para ajustar el modelo, realizar las predicciones y calcular las métricas de evaluación.

In [ ]:
lm = LinearRegression()
lm.fit(X_train.scaled, y_train)
print(lm)
print(lm.intercept_)
print(lm.coef_)

predictions = lm.predict(X_val.scaled)
predictions= predictions.reshape(-1,1)

plt.figure(figsize=(15,8))
plt.scatter(y_val, predictions)
plt.xlabel('Y Val')
plt.ylabel('Predicted Y')
plt.show()

print('MAE:', metrics.mean_absolute_error(y_val, predictions))
print('MSE:', metrics.mean_squared_error(y_val, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_val, predictions)))
print('R^2:', metrics.r2_score(y_val, predictions))

¿Observas diferencias en los coeficientes de la ecuación y/o en los resultados de las métricas de evaluación?

## Ejercicios propuestos

Al acabar el Notebook, puedes continuar trabajando con estos ejercicios:
- La librería sklearn incluye más modelos de regresión como `DecisionTreeRegressor`, `GradientBoostingRegressor`, o `RandomForestRegressor`. Prueba estos (y otros que quieras) y compara los resultados.
- Si ejecutaste el notebook de forma lineal, habrás calculado la regresión con el dataset con valores perdidos inputados. Prueba la estrategia alternativa de eliminar las filas que los contengan y compara los resultados.
- En este repositorio (https://github.com/mattnedrich/GradientDescentExample) hay un ejemplo de cálculo de regresión lineal mediante *gradient descent*, desarrollado en esta página: https://spin.atomicobject.com/2014/06/24/gradient-descent-linear-regression/ Echa un vistazo a la entrada de blog y, si tienes ganas y tiempo, programa el código y comprueba cómo funciona el algoritmo en un dataset sencillo.